In [2]:
import os
import getpass

from dotenv import load_dotenv

In [ ]:
# Set environment variable if not already set
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}")

In [4]:
# Load environment variables from .env file
load_dotenv()

api_list = ["GROQ_API_KEY", "TAVILY_API_KEY", "LANGSMITH_API_KEY"]

for api in api_list:
    if os.getenv(api) is None:
        print(f"{api} is not set")
        _set_env(api)
    else:
        print(f"{api} is set")

GROQ_API_KEY is set
TAVILY_API_KEY is set
LANGSMITH_API_KEY is set
